## Collected Mututal fund data using screeners

https://finance.yahoo.com/screener/unsaved/b8a2572f-78a8-407b-b317-827a6dde3238?count=100&offset=300



In [4]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

In [5]:
data = pd.read_csv('Yahoo_tickers_MF.txt',delimiter='\t')

In [41]:
data['Name'].loc[data['Name'].str.contains('Nippon')]

40      Nippon India Money Market Fund Weekly Dividend...
41      Nippon India Money Market Fund Monthly Dividen...
42      Nippon India Money Market Fund Direct Plan Bon...
43      Nippon India Money Market Fund Direct Plan Wee...
44      Nippon India Money Market Fund Monthly Dividen...
                              ...                        
9857       Nippon India Taiwan Equity Fund Regular Growth
9861    Nippon India Arbitrage Fund Direct Plan Monthl...
9897    Nippon India Overnight Fund Direct Quarterly D...
9921    Nippon India Nifty AAA PSU Bd Pl SDL Sep26 Mat...
9930    Nippon India Retirement Fund - Wealth Creation...
Name: Name, Length: 630, dtype: object

In [16]:
%matplotlib inline

import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np 
from ipywidgets import interact
from ipywidgets import interact



# MF_select = widgets.Dropdown(
#     options=data['Name'].tolist(),
#     value=data['Name'].iloc[0],
#     description='Mutual Fund',
#     disabled=False,
# )
# # Define a function to handle the change event
# def on_dropdown_change(change):
#     selected_option = change['new']
#     ticker = yf.Ticker(data.loc[data['Name'] == selected_option, 'Symbol'].iloc[0])
#     mf_df = ticker.history(period="5y")
#     #mf_df['Close'].plot(ax=ax,title=f"{selected_option} Stock Price")
#     print(mf_df.columns)
#     line, = ax.plot(mf_df.index,mf_df['Close'])
#     line.set_ydata(mf_df['Close'])
#     fig.canvas.draw()
# # Register the change event handler
# MF_select.observe(on_dropdown_change, names='value')


def fta(selected_option ='Nippon India Taiwan Equity Fund Regular Growth'):
    "showing sine frequency"
    ticker = yf.Ticker(data.loc[data['Name'] == selected_option, 'Symbol'].iloc[0])
    mf_df = ticker.history(period="5y")
    f, ax = plt.subplots(nrows=1,figsize=(8,6))
    #mf_df['Close'].plot(ax=ax,title=f"{selected_option} Stock Price")
    print(mf_df.columns)
    ax.plot(mf_df.index,mf_df['Close'])
    ax.grid();

df = interact( fta, Mutual_Fund=data['Name'].to_dict())



interactive(children=(Text(value='Nippon India Taiwan Equity Fund Regular Growth', description='selected_optio…

<function __main__.fta(selected_option='Nippon India Taiwan Equity Fund Regular Growth')>

In [13]:
interact??

Signature:      interact(_InteractFactory__interact_f=None, **kwargs)
Type:           _InteractFactory
String form:    <ipywidgets.widgets.interaction._InteractFactory object at 0x7fca2841f590>
File:           ~/miniconda3/envs/Finance/lib/python3.11/site-packages/ipywidgets/widgets/interaction.py
Source:        
class _InteractFactory:
    """
    Factory for instances of :class:`interactive`.

    This class is needed to support options like::

        >>> @interact.options(manual=True)
        ... def greeting(text="World"):
        ...     print("Hello {}".format(text))

    Parameters
    ----------
    cls : class
        The subclass of :class:`interactive` to construct.
    options : dict
        A dict of options used to construct the interactive
        function. By default, this is returned by
        ``cls.default_options()``.
    kwargs : dict
        A dict of **kwargs to use for widgets.
    """
    def __init__(self, cls, options, kwargs={}):
        self.cls = cls
    

In [10]:
# load the interactive tool
from ipywidgets import interact


import numpy as np
import matplotlib.pyplot as plt
x = np.linspace(0, 2 * np.pi, 1000)

def fta(freq = 50.0):
    "showing sine frequency"
    y = np.sin(freq*x)
    f, ax1 = plt.subplots(nrows=1,figsize=(8,6))
    ax1.plot(x[0:100], y[0:100],'b')
    ax1.set_ylim(ymin=-1.1, ymax=1.1)
    ax1.grid();
    # then use it interactively,
interact( fta, freq=(0.0,100.0))

interactive(children=(FloatSlider(value=50.0, description='freq'), Output()), _dom_classes=('widget-interact',…

<function __main__.fta(freq=50.0)>

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import numpy as np

out1 = widgets.Output()
out2 = widgets.Output()
data1 = pd.DataFrame(np.random.normal(size = 50))
data2 = pd.DataFrame(np.random.normal(size = 100))

tab = widgets.Tab(children = [out1, out2])
tab.set_title(0, 'First')
tab.set_title(1, 'Second')
display(tab)

with out1:
    fig1, axes1 = plt.subplots()
    data1.hist(ax = axes1)
    plt.show(fig1)

with out2:
    fig2, axes2 = plt.subplots()
    data2.hist(ax = axes2)
    plt.show(fig2)